<Font Size = 5> **Web Scraping**

In [1]:
# First I import the necessary packages
import pandas as pd
import sqlite3
import requests
import lxml.html

In [2]:
r = requests.get("https://www.infoplease.com/us/world-war-i-primer/casualties-world-war-i") #Now I pull the data from the website using request.

In [3]:
r

<Response [200]>

In [5]:
html                     = lxml.html.fromstring(r.text) 
ww1                      = html.get_element_by_id("A0004618")
tablebody                = ww1.getchildren()[0]

In [6]:
ww1.getchildren()

[<Element tbody at 0x123d09958>]

In [7]:
tablebody.getchildren()[0].getchildren() #I have my table or so I think, and use getchildren just to confirm.

[<Element th at 0x123d09c78>,
 <Element th at 0x123d09cc8>,
 <Element th at 0x123d09d18>,
 <Element th at 0x123d09d68>,
 <Element th at 0x123d09db8>,
 <Element th at 0x123d09e08>]

In [8]:
tablebody.getchildren()[2].text_content() #Another way to confirm I am working with the table.

'Belgium267,00013,71644,68634,65993,061'

<Font Size = 5> **Creating the Dataframe**

In [9]:
# at this point, I need to place my tabkle into a pandas dataframe so I can prepare to place the data into a sqlite3 database.

data = {
    "Country":                    [],
    "Total_mobilized_forces":     [],
    "Killed_or_died":             [],
    "Wounded":                    [],
    "Prisoners_or_missing":       [],
    "Total_casualties":           []
    
}


for row in tablebody.getchildren()[1:]:
    cells = row.getchildren()
    

    data["Country"].append(cells[0].text_content().strip())
    data["Total_mobilized_forces"].append(cells[1].text_content().strip())
    data["Killed_or_died"].append(cells[2].text_content().strip())
    data["Wounded"].append(cells[3].text_content().strip())
    data["Prisoners_or_missing"].append(cells[4].text_content().strip())
    data["Total_casualties"].append(cells[5].text_content().strip())
 
  
    
    
df = pd.DataFrame(data)
df

,Country,Total_mobilized_forces,Killed_or_died,Wounded,Prisoners_or_missing,Total_casualties
0,Austria-Hungary,"7,800,000","1,200,000","3,620,000","2,200,000","7,020,000"
1,Belgium,"267,000","13,716","44,686","34,659","93,061"
2,British Empire2,"8,904,467","908,371","2,090,212","191,652","3,190,235"
3,Bulgaria,"1,200,000","87,500","152,390","27,029","266,919"
4,France2,"8,410,000","1,357,800","4,266,000","537,000","6,160,800"
5,Germany,"11,000,000","1,773,700","4,216,058","1,152,800","7,142,558"
6,Greece,"230,000","5,000","21,000","1,000","27,000"
7,Italy,"5,615,000","650,000","947,000","600,000","2,197,000"
8,Japan,"800,000",300,907,3,"1,210"
9,Montenegro,"50,000","3,000","10,000","7,000","20,000"


<Font Size = 5> **Selecting Data to Plcae into the SQLITE3 Database**

In [10]:
for row in df[df.Country.isin(["United States", "Russia", "Austria-Hungary", "Belgium", "British Empire2", "France2", "Germany"])].itertuples():
    print(row[1:])

('Austria-Hungary', '7,800,000', '1,200,000', '3,620,000', '2,200,000', '7,020,000')
('Belgium', '267,000', '13,716', '44,686', '34,659', '93,061')
('British Empire2', '8,904,467', '908,371', '2,090,212', '191,652', '3,190,235')
('France2', '8,410,000', '1,357,800', '4,266,000', '537,000', '6,160,800')
('Germany', '11,000,000', '1,773,700', '4,216,058', '1,152,800', '7,142,558')
('Russia', '12,000,000', '1,700,000', '4,950,000', '2,500,000', '9,150,000')
('United States', '4,734,991', '116,516', '204,002', '—', '320,518')


In [11]:
len(row[1:])

6

In [12]:
#Creating and connecting to the database.


db = sqlite3.connect(":memory:")
cursor = db.cursor()
cursor.execute("""
CREATE TABLE WORLD_WAR_1(COUNTRY, TOTAL_MOBILIZED_FORCES, KILLED_OR_DIED, WOUNDED, PRISONERS_OR_MISSING, TOTAL_CASUALTIES)
""")

#I am only going to select some of the major players in World War One.

for row in df[df.Country.isin(["United_States", "Russia", "Austria-Hungary", "Belgium", "British_Empire2", "France2", "Germany"])].itertuples():
    
    insert_sql_syntax = """
    INSERT INTO WORLD_WAR_1(COUNTRY, TOTAL_MOBILIZED_FORCES, KILLED_OR_DIED, WOUNDED, PRISONERS_OR_MISSING, TOTAL_CASUALTIES)
    VALUES (?,?,?,?,?,?)
    """
    cursor.execute(insert_sql_syntax, row[1:] )
    
       
db.commit()

In [13]:
#Time to select data from the sqlite3 database I created to see if it worked.

for row in cursor.execute("""
    SELECT *
    FROM WORLD_WAR_1
    WHERE COUNTRY = "Russia"
"""):
    print(row)

('Russia', '12,000,000', '1,700,000', '4,950,000', '2,500,000', '9,150,000')


In [14]:
#Finally, I close my database so that it knows I am done working with it. 
db.close()